# Notebook 02: Treatment Design and Sample Selection


**Goal:** Create clean analysis sample, validate DiD assumptions, check covariate balance


## Setup


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(project_root)
from src.utils import create_pseudo_customer_id, plot_covariate_balance

pd.set_option('display.max_columns', None)

## 1. Load Data and Create Customer IDs


In [2]:
# Load wave-labeled data from notebook 01
df = pd.read_csv('data/processed/data_with_waves.csv')
df['contact_date'] = pd.to_datetime(df['contact_date'])

print(f"Total contact events: {len(df):,}")
print(f"Contacts in analysis waves: {len(df[df['wave'].isin(['wave_1', 'wave_2'])]):,}")

Total contact events: 41,188
Contacts in analysis waves: 33,142


### 1.1 Create Pseudo-Customer IDs


In [3]:
# Since dataset lacks customer IDs, create composite identifier
# using stable demographic attributes

df = create_pseudo_customer_id(df, id_cols=[
    'age', 'job', 'marital', 'education', 'housing', 'loan', 'contact'
])

print(f"\n{'='*60}")
print(f"CUSTOMER IDENTIFICATION")
print(f"{'='*60}")
print(f"Total contact rows: {len(df):,}")
print(f"Unique pseudo-customers: {df['pseudo_id'].nunique():,}")
print(f"Ratio: {df['pseudo_id'].nunique() / len(df):.1%}")
print(f"Avg contacts per customer: {len(df) / df['pseudo_id'].nunique():.1f}")


CUSTOMER IDENTIFICATION
Total contact rows: 41,188
Unique pseudo-customers: 14,010
Ratio: 34.0%
Avg contacts per customer: 2.9


**Expected output:** ~34% unique IDs, ~3 contacts per customer

## 2. Check Cross-Wave Contamination


In [5]:
# Identify customers contacted in both waves
df_waves = df[df['wave'].isin(['wave_1', 'wave_2'])]

customer_waves = df_waves.groupby('pseudo_id')['wave'].nunique()

print(f"\n{'='*60}")
print(f"CROSS-WAVE ANALYSIS")
print(f"{'='*60}")
print(f"Total customers in analysis waves: {len(customer_waves):,}")
print(f"Single-wave customers: {(customer_waves == 1).sum():,} ({(customer_waves == 1).mean():.1%})")
print(f"Cross-wave customers: {(customer_waves > 1).sum():,} ({(customer_waves > 1).mean():.1%})")

if (customer_waves > 1).sum() > 0:
    print("\n⚠️  WARNING: Cross-wave contamination detected!")
    print("These customers cannot be cleanly assigned to treatment or control.")
    print("Decision: Exclude cross-wave customers from analysis.")


CROSS-WAVE ANALYSIS
Total customers in analysis waves: 11,970
Single-wave customers: 9,858 (82.4%)
Cross-wave customers: 2,112 (17.6%)

⚠️  WARNING: Cross-wave contamination detected!
These customers cannot be cleanly assigned to treatment or control.
Decision: Exclude cross-wave customers from analysis.


## 3. Create Customer-Level Dataset


In [6]:
# Keep single-wave customers only
single_wave_customers = customer_waves[customer_waves == 1].index

df_contacts = df[
    df['pseudo_id'].isin(single_wave_customers) & 
    df['wave'].isin(['wave_1', 'wave_2'])
].copy()

print(f"\nContact-level dataset (single-wave customers only):")
print(f"Total contacts: {len(df_contacts):,}")
print(f"Unique customers: {df_contacts['pseudo_id'].nunique():,}")
print(f"Wave 1 contacts: {(df_contacts['wave']=='wave_1').sum():,}")
print(f"Wave 2 contacts: {(df_contacts['wave']=='wave_2').sum():,}")


Contact-level dataset (single-wave customers only):
Total contacts: 18,291
Unique customers: 9,858
Wave 1 contacts: 15,001
Wave 2 contacts: 3,290


### 3.1 Keep First Contact Per Customer


In [7]:
# For clean DiD, use customer-level data (first contact only)
df_customer = df_contacts.sort_values('contact_date').groupby('pseudo_id').first().reset_index()

print(f"\n{'='*60}")
print(f"CUSTOMER-LEVEL DATASET (First Contact Per Customer)")
print(f"{'='*60}")
print(f"Total customers: {len(df_customer):,}")
print(f"Wave 1: {(df_customer['wave']=='wave_1').sum():,}")
print(f"Wave 2: {(df_customer['wave']=='wave_2').sum():,}")

# Verify uniqueness
assert df_customer['pseudo_id'].nunique() == len(df_customer), "ERROR: Duplicate customers!"
print("✅ Verification passed: Each customer appears exactly once")


CUSTOMER-LEVEL DATASET (First Contact Per Customer)
Total customers: 9,858
Wave 1: 7,475
Wave 2: 2,383
✅ Verification passed: Each customer appears exactly once


## 4. Check Prior Contact History


In [8]:
# Investigate 'previous' and 'pdays' variables
# previous = number of contacts in prior campaigns
# pdays = days since last contact (999 = never contacted)

print(f"\n{'='*60}")
print(f"PRIOR CONTACT HISTORY")
print(f"{'='*60}")

# Wave 1 composition
print("\nWave 1:")
print(f"  pdays == 999: {(df_customer[df_customer['wave']=='wave_1']['pdays'] == 999).sum():,} "
      f"({(df_customer[df_customer['wave']=='wave_1']['pdays'] == 999).mean():.1%})")
print(f"  previous == 0: {(df_customer[df_customer['wave']=='wave_1']['previous'] == 0).sum():,} "
      f"({(df_customer[df_customer['wave']=='wave_1']['previous'] == 0).mean():.1%})")

# Wave 2 composition
wave_2_df = df_customer[df_customer['wave']=='wave_2']
print("\nWave 2:")
print(f"  pdays == 999: {(wave_2_df['pdays'] == 999).sum():,} ({(wave_2_df['pdays'] == 999).mean():.1%})")
print(f"  previous == 0: {(wave_2_df['previous'] == 0).sum():,} ({(wave_2_df['previous'] == 0).mean():.1%})")

# Crosstab
ct = pd.crosstab(
    wave_2_df['pdays'] == 999,
    wave_2_df['previous'] == 0,
    margins=True
)
print("\nWave 2 Crosstab (pdays==999 vs previous==0):")
print(ct)


PRIOR CONTACT HISTORY

Wave 1:
  pdays == 999: 7,475 (100.0%)
  previous == 0: 7,475 (100.0%)

Wave 2:
  pdays == 999: 2,298 (96.4%)
  previous == 0: 1,650 (69.2%)

Wave 2 Crosstab (pdays==999 vs previous==0):
previous  False  True   All
pdays                      
False        85     0    85
True        648  1650  2298
All         733  1650  2383


### 4.1 Subscription Rates by Prior Contact Status


In [9]:
# Check if prior contact predicts outcome
print(f"\n{'='*60}")
print(f"SUBSCRIPTION RATES BY PRIOR CONTACT")
print(f"{'='*60}")

for wave in ['wave_1', 'wave_2']:
    wave_df = df_customer[df_customer['wave']==wave]
    
    print(f"\n{wave.upper()}:")
    
    # By previous
    prev_rates = wave_df.groupby('previous')['y_binary'].agg(['mean', 'count'])
    print("\nBy 'previous' (count of prior contacts):")
    print(prev_rates.head())
    
    # By pdays
    pdays_fresh = wave_df[wave_df['pdays'] == 999]['y_binary'].mean()
    pdays_contacted = wave_df[wave_df['pdays'] < 999]['y_binary'].mean() if (wave_df['pdays'] < 999).any() else None
    
    print(f"\nBy 'pdays':")
    print(f"  pdays == 999 (never contacted): {pdays_fresh:.1%}")
    if pdays_contacted:
        print(f"  pdays < 999 (previously contacted): {pdays_contacted:.1%}")


SUBSCRIPTION RATES BY PRIOR CONTACT

WAVE_1:

By 'previous' (count of prior contacts):
             mean  count
previous                
0         0.04388   7475

By 'pdays':
  pdays == 999 (never contacted): 4.4%

WAVE_2:

By 'previous' (count of prior contacts):
              mean  count
previous                 
0         0.168485   1650
1         0.121563    691
2         0.179487     39
3         0.333333      3

By 'pdays':
  pdays == 999 (never contacted): 14.5%
  pdays < 999 (previously contacted): 43.5%


## 5. Final Sample Selection: Fresh Prospects Only


In [10]:
# DECISION: Restrict to truly fresh prospects
# - pdays == 999 (never contacted before)
# - previous == 0 (zero prior contacts)
# This ensures clean apples-to-apples comparison

df_final = df_customer[
    (df_customer['pdays'] == 999) & 
    (df_customer['previous'] == 0)
].copy()

print(f"\n{'='*60}")
print(f"FINAL ANALYSIS SAMPLE")
print(f"{'='*60}")
print(f"Total customers: {len(df_final):,}")
print(f"Wave 1 (May-Aug 2008): {(df_final['wave']=='wave_1').sum():,}")
print(f"Wave 2 (Apr-Aug 2009): {(df_final['wave']=='wave_2').sum():,}")
print(f"\nSample restriction:")
print(f"  - Excluded cross-wave customers: {(customer_waves > 1).sum():,}")
print(f"  - Excluded customers with prior contact: "
      f"{len(df_customer) - len(df_final):,}")


FINAL ANALYSIS SAMPLE
Total customers: 9,125
Wave 1 (May-Aug 2008): 7,475
Wave 2 (Apr-Aug 2009): 1,650

Sample restriction:
  - Excluded cross-wave customers: 2,112
  - Excluded customers with prior contact: 733


## 6. Covariate Balance Check


In [11]:
# Check if treatment and control groups are comparable
# on observable characteristics

covariates = [
    'age', 'campaign',  # Customer characteristics
    'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',  # Economic indicators
    'euribor3m', 'nr.employed'
]

balance = df_final.groupby('wave')[covariates].mean().T
balance.columns = ['Wave 1', 'Wave 2']
balance['Difference'] = balance['Wave 1'] - balance['Wave 2']
balance['Abs % Diff'] = (abs(balance['Difference']) / abs(balance['Wave 1']) * 100).round(1)

print(f"\n{'='*60}")
print(f"COVARIATE BALANCE")
print(f"{'='*60}")
print(balance.round(2))


COVARIATE BALANCE
                 Wave 1   Wave 2  Difference  Abs % Diff
age               41.98    40.49        1.48         3.5
campaign           2.94     2.32        0.62        21.0
emp.var.rate       1.26    -1.91        3.17       251.4
cons.price.idx    93.99    92.95        1.03         1.1
cons.conf.idx    -38.92   -45.82        6.90        17.7
euribor3m          4.91     1.32        3.59        73.1
nr.employed     5210.85  5096.85      114.00         2.2


### 6.1 Visualize Balance


In [12]:
# Separate customer characteristics from economic indicators
cust_vars = ['age', 'campaign']
econ_vars = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

# Plot customer characteristics
fig_cust = plot_covariate_balance(
    balance.loc[cust_vars],
    title="Customer Characteristics Balance (Should Be <10%)"
)
fig_cust.show()

# Economic indicators (expect large differences)
fig_econ = go.Figure()

fig_econ.add_trace(go.Bar(
    name='Wave 1 (2008)',
    x=econ_vars,
    y=balance.loc[econ_vars, 'Wave 1'],
    marker_color='lightblue'
))

fig_econ.add_trace(go.Bar(
    name='Wave 2 (2009)',
    x=econ_vars,
    y=balance.loc[econ_vars, 'Wave 2'],
    marker_color='salmon'
))

fig_econ.update_layout(
    title='Economic Indicators: Dramatic Divergence (GOOD for DiD)',
    yaxis_title='Mean Value',
    barmode='group',
    height=400
)
fig_econ.show()

**Assessment:**
- **Customer characteristics similar** (<10% difference) → No selection bias
- **Economic indicators diverge dramatically** → Strong quasi-experimental variation

## 7. Subscription Rate Comparison


In [13]:
# Naive comparison (unadjusted for confounders)

print(f"\n{'='*60}")
print(f"SUBSCRIPTION RATES (Fresh Prospects Only)")
print(f"{'='*60}")

rates = df_final.groupby('wave')['y_binary'].agg(['mean', 'count', 'sum'])
rates.index = ['Wave 1 (May-Aug 2008)', 'Wave 2 (Apr-Aug 2009)']
rates.columns = ['Subscription Rate', 'N Customers', 'N Subscribed']

print(rates)

# Calculate difference
diff = (rates.loc['Wave 2 (Apr-Aug 2009)', 'Subscription Rate'] - 
        rates.loc['Wave 1 (May-Aug 2008)', 'Subscription Rate'])

print(f"\nNaive difference: {diff:.4f} ({diff*100:.2f} percentage points)")
print(f"Relative increase: {(diff / rates.loc['Wave 1 (May-Aug 2008)', 'Subscription Rate']) * 100:.0f}%")

# Statistical test
w1_outcomes = df_final[df_final['wave']=='wave_1']['y_binary']
w2_outcomes = df_final[df_final['wave']=='wave_2']['y_binary']
t_stat, p_val = stats.ttest_ind(w1_outcomes, w2_outcomes)

print(f"\nT-test:")
print(f"  t-statistic: {t_stat:.3f}")
print(f"  p-value: {p_val:.4f}")
print(f"  Significant at 5%: {'Yes ✅' if p_val < 0.05 else 'No ❌'}")


SUBSCRIPTION RATES (Fresh Prospects Only)
                       Subscription Rate  N Customers  N Subscribed
Wave 1 (May-Aug 2008)           0.043880         7475           328
Wave 2 (Apr-Aug 2009)           0.168485         1650           278

Naive difference: 0.1246 (12.46 percentage points)
Relative increase: 284%

T-test:
  t-statistic: -18.747
  p-value: 0.0000
  Significant at 5%: Yes ✅


### 7.1 Visualize Outcome Difference


In [14]:
# Create comparison plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=['Wave 1<br>(Pre-Crisis)', 'Wave 2<br>(Crisis Recovery)'],
    y=[
        rates.loc['Wave 1 (May-Aug 2008)', 'Subscription Rate'] * 100,
        rates.loc['Wave 2 (Apr-Aug 2009)', 'Subscription Rate'] * 100
    ],
    text=[f"{r*100:.1f}%" for r in rates['Subscription Rate']],
    textposition='outside',
    marker_color=['lightblue', 'salmon']
))

fig.add_annotation(
    x=0.5, y=max(rates['Subscription Rate'] * 100) * 0.5,
    text=f"Δ = +{diff*100:.1f}pp<br>(+{(diff / rates.iloc[0,0])*100:.0f}%)",
    showarrow=False,
    font=dict(size=16, color="green"),
    bgcolor="lightyellow",
    opacity=0.8
)

fig.update_layout(
    title='Subscription Rates: Fresh Prospects Only',
    yaxis_title='Subscription Rate (%)',
    height=400,
    showlegend=False
)

fig.show()

## 8. Economic Story


In [15]:
print(f"\n{'='*60}")
print(f"ECONOMIC CONTEXT")
print(f"{'='*60}")

econ_context = balance.loc[econ_vars, ['Wave 1', 'Wave 2', 'Difference']]

print("\nMacroeconomic conditions between campaign waves:")
print(econ_context.round(2))

print("\nInterpretation:")
print("- Employment variation rate: Swung from growth (+1.3%) to recession (-1.9%)")
print("- Euribor 3-month rate: Crashed 73% (4.9% → 1.3%)")
print("- Consumer confidence: Deteriorated (-38.9 → -45.8)")
print("- Employment level: Lost 114,000 jobs")
print("\nCounterfactual question:")
print("Would subscription rates have increased WITHOUT the economic shock?")
print("DiD methodology will help us answer this.")


ECONOMIC CONTEXT

Macroeconomic conditions between campaign waves:
                 Wave 1   Wave 2  Difference
emp.var.rate       1.26    -1.91        3.17
cons.price.idx    93.99    92.95        1.03
cons.conf.idx    -38.92   -45.82        6.90
euribor3m          4.91     1.32        3.59
nr.employed     5210.85  5096.85      114.00

Interpretation:
- Employment variation rate: Swung from growth (+1.3%) to recession (-1.9%)
- Euribor 3-month rate: Crashed 73% (4.9% → 1.3%)
- Consumer confidence: Deteriorated (-38.9 → -45.8)
- Employment level: Lost 114,000 jobs

Counterfactual question:
Would subscription rates have increased WITHOUT the economic shock?
DiD methodology will help us answer this.


## 9. Save Final Analysis Sample


In [17]:
# Save clean dataset for DiD estimation
df_final.to_csv('data/processed/analysis_sample.csv', index=False)

print(f"\n✅ Saved final analysis sample:")
print(f"   Location: data/processed/analysis_sample.csv")
print(f"   N = {len(df_final):,} customers")
print(f"   Wave 1: {(df_final['wave']=='wave_1').sum():,}")
print(f"   Wave 2: {(df_final['wave']=='wave_2').sum():,}")


✅ Saved final analysis sample:
   Location: data/processed/analysis_sample.csv
   N = 9,125 customers
   Wave 1: 7,475
   Wave 2: 1,650
